%%html
<style>
div.input {
    display:none;
}
</style>

In [60]:
from catboost import CatBoostClassifier
from ipywidgets import widgets, Layout
from pattern.text.nl import sentiment #sentiment analysis
import pandas as pd
import numpy as np
import json

In [61]:
%%capture
model1 = CatBoostClassifier()
model1.load_model(fname = './models/model1-33_BEST.cbm', format = 'cbm')

#model2 = CatBoostClassifier()
#model2.load_model(fname = '/models/model2-33_BEST.cbm', format = 'cbm')

#model3 = CatBoostClassifier()
#model3.load_model(fname = '/models/model3-33_BEST.cbm', format = 'cbm')

#model4 = CatBoostClassifier()
#model4.load_model(fname = '/models/model4-33_BEST.cbm', format = 'cbm')

#model5 = CatBoostClassifier()
#model5.load_model(fname = '/models/model5-33_BEST.cbm', format = 'cbm')

In [62]:
#Source: https://stackoverflow.com/questions/43146528/how-to-extract-all-the-emojis-from-text
import regex
def emojis_str(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return ''.join(emoji_list)

In [63]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import regex
import time
try:
    data = json.load(open("hashtags.json", "r"))
    df = pd.DataFrame.from_dict(data, orient="index")
    dictOfHashtags = df.to_dict('dict')[0]
except:
    dictOfHashtags = {}
    
def hashtags(dict_, dictOfHashtags):
    
    dt = dict_['date']
    #print(dt)
    try:
        string = dictOfHashtags[dt]
    except:
        url = 'https://trendogate.com/placebydate/23424757/'+dt
        #print(url)
        response = requests.get(url)
        tx = BeautifulSoup(response.text, 'html.parser')
        string = ""
        #flag = true
        ind = tx.find("h2",text=" Trends in Belgium / Belgium ")
        while ind:
            ind = ind.find_next("li")
            try:
                string = string + ind.text
            except:
                #print("end of hashtags")
                pass
        string = regex.sub('#', '', string)
        dictOfHashtags[dt]=string
        time.sleep(1)
    count = 0
    try:
        for el in dict_['hashtags']:
            #print(el['text'])
            if el in string: 
                count = count + 1
    except:
        print("In function hashtags: exception for:"+dict_['id_str'])
    #print(string)
    #print (str(count)+'\n \n')
    return count

In [64]:
from datetime import date
def on_button_clicked(b, model1 = model1):#, model2 = model2, model3 = model3, model4 = model4, model5 = model5):
    qual.description = 'please wait'
    qual.button_style = 'info'
    try:
        import emoji
        emojis = emojis_str(text)
        num_emojis = len(emojis)
        emoji_polarity = 0
    except:
        num_emojis = 0
        emoji_polarity = 0

    try:
        from pattern.text.nl import sentiment #sentiment analysis
        result = sentiment(dict_['tweet_text'])
        polarity = result[0]
        subjectivity = result[1]
    except:
        polarity = 0
        subjectivity = 0

    text = tweet_text.value
    num_urls = text.count("http")-quote.value
    num_mentions = text.count("@")
    num_hashtags = text.count("#")
    length = len(text)
           
    reg = regex.compile(r"#(\w+)")
    tags = reg.findall(text)
    dat = date_picker.value
    
    try:
        dict_ = {'date': dat.strftime("%Y-%m-%d"), 'hashtags': tags}
        num_hashtags_trending = hashtags(dict_, dictOfHashtags)
    except:
        print('Please fill in a date!')
        
    delta = dat-date.today() 
    days_since_post = -delta.days
    day_week = dat.weekday()
       
    data = {'num_hashtags': num_hashtags,
            'num_mentions': num_mentions,
            'num_urls': num_urls,
            'num_emojis': num_emojis,
            'emoji_polarity': emoji_polarity,
            'subjectivity': subjectivity,
            'polarity': polarity,
            'length': length,
            'reply': reply.value,
            'quote': quote.value,
            'category': 'andere',
            'screen_name': party.value,
            'media_type': media.value,
            'days_since_post': days_since_post,
            'time_day': hour.value,
            'day_week': day_week,
            'num_hashtags_trending': num_hashtags_trending
           } 
  
    x = pd.DataFrame(data,  index=[0]) 
        
    y_pred1 = model1.predict(x)
    #y_pred2 = model2.predict(x)
    #y_pred3 = model3.predict(x)
    #y_pred4 = model4.predict(x)
    #y_pred5 = model5.predict(x)
    
    #y_comp = pd.concat([pd.Series(np.ravel(y_pred1)), pd.Series(np.ravel(y_pred2)), pd.Series(np.ravel(y_pred3)), pd.Series(np.ravel(y_pred4)),pd.Series(np.ravel(y_pred5))],axis=1)
    #y_mean = y_comp.mode(axis=1).iloc[:, 0] #choose prediction that is most frequent per tweet
    y_mean = y_pred1[0]
    qual.description = y_mean[0]
    if y_mean[0] == 'bad':
        qual.button_style = 'danger'
    elif y_mean[0] == 'medium':
        qual.button_style = 'warning'
    else:
        qual.button_style = 'success'

In [65]:
tweet_text = widgets.Text(description='Tweet text:',)
media = widgets.Dropdown(
    options=['video', 'photo', 'none'],
    value='none',
    description='Media Type:',
)

party = widgets.Dropdown(
    options=['pvdabelgie', 'groen', 'sp_a', 'cdenv', 'openvld', 'de_NVA', 'vlbelang'],
    value='pvdabelgie',
    description='Party:',
)
reply = widgets.Checkbox(
    value=False,
    description='Reply',
)
quote = widgets.Checkbox(
    value=False,
    description='Quote',
)
hour = widgets.IntSlider(
    value=12,
    min=0,
    max=24,
    step=1,
    description='Hour:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

date_picker = widgets.DatePicker(
    description='Date:',
)
{
    "tags": [
        "hide_input",
    ]
}
type_tweet = widgets.RadioButtons(
    options=['normal', 'reply', 'quote'],
    value='normal',
    description='Percentage of tweets to be counted as bad/medium/good',
)

BMG = widgets.RadioButtons(
    options=['50/25/25', '33/33/33'],
    value='50/25/25',
    description='Percentage of tweets to be counted as bad/medium/good',
)
submit = widgets.Button(
    description='Submit',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check',
    layout = Layout(align_items = 'center')
)

qual = widgets.Button(
    description='Tweet Quality',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout = Layout(align_items = 'center')
)

box_layout = widgets.Layout(display='flex',
               flex_flow='row',
               align_items='center',
                width='50%')
hbox = widgets.HBox(children=[submit, qual],layout=box_layout)
submit.on_click(on_button_clicked)
box = widgets.VBox([tweet_text, media, party, reply, quote, hour, date_picker, BMG, hbox])
box

